# Post Call Analytics
Welcome to this training module on post-call analytics use cases using Amazon SageMaker JumpStart.

As businesses continue to interact with customers through various channels, it becomes increasingly important to analyze these interactions to gain insights into customer behavior and preferences. Post-call analytics is one such method that involves analyzing customer interactions after the call has ended. The use of large language models can greatly enhance the effectiveness of post-call analytics by enabling more accurate sentiment analysis, identifying specific customer needs and preferences, and improving overall customer experience.

In this sample notebook, we will explore following topics to demonstrate the various benefits of using Bedrock for post-call analytics and businesses gain a competitive edge in the modern marketplace.

* One model handling multiple PCA tasks <BR>
* Handling long call transcripts <BR>

## Step 0. Install packages

In [ ]:
install_needed = True  # should only be True once

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U transformers
    
    IPython.Application.instance().kernel.do_shutdown(True)

## Step 1. SageMaker Endpoint Wrapper

In [ ]:
import json
import boto3
from pprint import pprint
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint

In [ ]:
class VarcoContentHandler(LLMContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        '''
        입력 데이터 전처리 후에 리턴
        '''
        
        payload = {"text": prompt}
        payload.update(model_kwargs)
        input_str = json.dumps(payload)

        return input_str.encode('utf-8')

    def transform_output(self, output: bytes) -> str:
    
        response_json = json.loads(output.read().decode("utf-8"))     
        generated_text = response_json["result"]        

        return generated_text[0]

In [ ]:
aws_region = boto3.Session().region_name
llm_text_content_handler = VarcoContentHandler()
endpoint_name_text = "varco-llm-13b-ist"

In [ ]:
params = {
    "request_output_len": 256,
    "repetition_penalty": 1.15,
    "temperature": 0.1,
    "top_k": 50,
    "top_p": 1
}

llm_text = SagemakerEndpoint(
    endpoint_name=endpoint_name_text,
    region_name=aws_region,
    model_kwargs=params,
    content_handler=llm_text_content_handler,
)

## Step 2. Load transcript files

In [ ]:

transcript_files = [
    "./call_transcripts/negative-refund-ko.txt",
    "./call_transcripts/neutral-short-ko.txt",
    "./call_transcripts/positive-partial-refund-ko.txt",
    "./call_transcripts/aws-short-ko.txt",
]
transcripts = []

for file_name in transcript_files:
    with open(file_name, "r") as file:
        transcripts.append(file.read())


for i, trans in enumerate(transcripts):
    print(f"transcript #{i+1}: {trans[:300]}\n")
    print("====================\n\n")

## Step 3. Post Call Analysis

### Step 3.1. Prompt Template
In this notebook, we'll be performing four different analyses(Summary, Sentiment, Intent and Resolution), and we'll need a template for each one.

* Summary template

In [ ]:
summary_template = """### User:
다음 대화를 간단하게 요약해 주세요.
대화: {transcript}


### Assistant:
"""

* Sentiment template

In [ ]:
sentiment_template = """### User:
감성 분석 프로그램입니다. 다음 클래스를 이용하여 고객의 감성을 분류하세요. 
["긍정", "중립", "부정"]. 대화를 이 클래스 중 한 가지로 정확하게 분류합니다. 
모르거나 확실하지 않은 경우 ["중립"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.
대화: {transcript}


### Assistant:
"""

* intent template

In [ ]:
intent_template = """### User:
이것은 의도 분류 프로그램입니다. 다음 대화에서 고개의 목적은 무엇입니까? 
클래스 ["배송_지연", "제품_결함", "계정_질문"]. 대화를 다음 클래스 중 하나로 분류합니다. 
이 클래스 중 하나에 정확히 일치합니다. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요. 
대화: {transcript}


### Assistant:
"""

### Step 3.2. Analysis

In [ ]:
from termcolor import colored
from langchain import PromptTemplate

In [ ]:
def analysis(llm, transcript, params, template="", max_tokens=50):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    analysis_prompt = prompt.format(transcript=transcript)
    llm.model_kwargs = params

    print (colored(analysis_prompt, 'green'))

    response = llm(analysis_prompt)

    return response

In [ ]:
params = {
    "request_output_len": 256,
    "repetition_penalty": 0.8,
    "temperature": 0.9,
    "top_k": 50,
    "top_p": 1
}

* summary

In [ ]:
%%time

res = analysis(
    llm=llm_text,
    transcript=transcripts[0],
    params=params,
    template=summary_template
)

print (res)

* sentiment

In [ ]:
%%time

res = analysis(
    llm=llm_text,
    transcript=transcripts[0],
    params=params,
    template=sentiment_template
)

print (res)

* intent

In [ ]:
%%time

res = analysis(
    llm=llm_text,
    transcript=transcripts[0],
    params=params,
    template=intent_template
)

print (res)

## Handling long call transcripts
We'll cover how to handle long transcripts that exceed the limits of the LLM.

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

* prompting to divide and conquer

In [ ]:
stuff_prompt_template = """### User:
다음 글을 간단하게 요약해 주세요.
글: {text}

### Assistant:
"""

chuck_prompt_template = """### User:
다음 글을 간단하게 요약해 주세요.
글: {text}

### Assistant:
"""

chunk_prompt = PromptTemplate(
    template=chuck_prompt_template,
    input_variables=["text"]
)

combine_prompt_template = """### User:
다음 글을 간단하게 요약해 주세요.
글: {text}


### Assistant:
"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template,
    input_variables=["text"]
)

* summarize chain

In [ ]:
'''
# summary_chain = load_summarize_chain(
#     llm=llm,
#     chain_type="map_reduce",
#     verbose=True
# ) # map_reduce, refine
# transcript = summary_chain(docs)
'''


def summary_chain_init(chain_type, llm):
    
    if chain_type == "STUFF":
        chain = load_summarize_chain(
            llm,
            chain_type="stuff",
            verbose=True
        )
        
    elif chain_type == "MAP_REDUCE":
        chain = load_summarize_chain(
            llm,
            chain_type="map_reduce",
            map_prompt=chunk_prompt,
            combine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
    elif chain_type == "REFINE":
        chain = load_summarize_chain(
            llm,
            chain_type="refine",
            question_prompt=chunk_prompt,
            refine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
        
    return chain

In [ ]:
def long_call_analysis(llm, transcript, params, template="", chain_type="MAP_REDUCE", max_tokens=50):

    
    llm.model_kwargs = params
    num_tokens = llm.get_num_tokens(transcript) #raise warnning

    if num_tokens > max_tokens:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n\n"],
            chunk_size=500,
            chunk_overlap=100
        )
        docs = text_splitter.create_documents([transcript])
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

        
        summary_chain = summary_chain_init(
            chain_type=chain_type, 
            llm=llm
        )
        response = summary_chain(
            {"input_documents": docs}
        )
        
        print ("Intermediate_steps: \n")
        for idx, step in enumerate(response["intermediate_steps"]):
            print (colored(f'step {idx}: \n', "green"))
            print (colored(f'{step}\n', "green"))
        
        return response["output_text"]
    
    else:
        
        prompt = PromptTemplate(template=stuff_prompt_template, input_variables=["text"])
        analysis_prompt = prompt.format(text=transcript)
        print (colored(analysis_prompt, 'green'))
        
        response = llm(analysis_prompt)
        
        return response
        

In [ ]:
params = {
    "repetition_penalty": 1.01,
    "temperature": 0.1,
    "top_k": 50,
    "top_p": 0.9
}

In [ ]:

%%time

res = long_call_analysis(
    llm=llm_text,
    transcript=transcripts[3],
    params=params,
    template=summary_template,
    chain_type="REFINE" # REFINE, MAP_REDUCE
)

print ("Results: \n")
print (res)